<center><h1>Image regeneration using GA</h1></center>

### 0. Importing all the necessary libraries

In [2]:
import re
import os
import cv2
import shutil
import random
import imageio
import operator
import functools
import itertools
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import randint

### 1. Defining global constants

In [3]:
qualities = []
num_parents_mating = 3
mutation_percent = 2
sol_per_pop = 8

### 2. Reading the image for experiment

In [4]:
face = cv2.imread('face.png')
face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
M, N = face.shape[:2]

### 3. Data representation

In [5]:
def img2vector(img):
    return np.reshape(a=img, newshape=(functools.reduce(operator.mul, img.shape)))

In [6]:
def convert2img(chromo, imgShape):
    return np.reshape(a=chromo, newshape=imgShape)

### 4. Generating initial population

In [7]:
def initialPopulation(imgShape, nIndividuals=8):
    init_population = np.empty(shape=(nIndividuals, functools.reduce(operator.mul, imgShape)), dtype = np.uint8)
    for indv_num in range(nIndividuals):
        init_population[indv_num, :] = randint(0,2,M*N)
    return init_population

### 5. Calculating fitness

In [8]:
def fitness_func(target, chromo):
    return -1*np.sum(np.abs(chromo-target))

def calPopFitness(target,pop):
    qualities = np.zeros(pop.shape[0])
    for num in range(pop.shape[0]):
        qualities[num] = fitness_func(target, pop[num, :])
    return qualities

### 6. Selection

In [9]:
def getBestParents(population,qualities,n=4):
    parents = []
    i = np.argsort(qualities)[::-1]
    n_fit=np.sort(qualities)[::-1]
    print('Index of best -{}'.format(i))
    print('Best qualites -{}'.format(n_fit))
    best_parent_index = i[0:n]
    for j in best_parent_index:
        parents.append(population[j])
    parents=np.array(parents)
    return parents

### 7. Crossover

In [10]:
def crossover(parents, imgShape, nIndividuals=8):
    new_population = np.empty(shape=(nIndividuals,functools.reduce(operator.mul, imgShape)),dtype = np.uint8)
    new_population[0:parents.shape[0], :] = parents
    num_newly_generated = nIndividuals-parents.shape[0]
    parents_permutations = list(itertools.permutations(iterable=np.arange(0, parents.shape[0]),r=2))
    selected_permutations = random.sample(range(len(parents_permutations)),num_newly_generated)
    comb_idx = parents.shape[0]
    for comb in range(len(selected_permutations)):
        selected_comb_idx = selected_permutations[comb]
        selected_comb = parents_permutations[selected_comb_idx]
        cross_size = np.int32(new_population.shape[1]/50)
        new_population[comb_idx+comb, 0:cross_size] = parents[selected_comb[0],0:cross_size]
        new_population[comb_idx+comb, cross_size:] = parents[selected_comb[1],cross_size:]
    return new_population

### 8. Mutation

In [11]:
def mutation(population, numParentsMating, mutPercent):
    for i in range(numParentsMating, population.shape[0]):
        rand_indx = np.uint32(np.random.random(size=np.uint32(mutPercent/100*population.shape[1]))*population.shape[1])
        new_values = randint(0, 2, len(rand_indx))
        population[i, rand_indx] = new_values
    return population

### 9. Some utility functions

In [12]:
def flush_dir(directory):
    shutil.rmtree(directory)
    os.makedirs(directory) 

In [13]:
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

In [14]:
def saveImages(currIteration, qualities, population, imShape, save_point, save_dir):
    if(np.mod(currIteration+1, save_point)==0):
        best_solution_chrom = population[np.where(qualities == np.max(qualities))[0][0],:]
        best_solution_img = convert2img(best_solution_chrom, imShape)
        fig,a = plt.subplots(1,2)
        fig.suptitle(f'Iteration {currIteration+1}')
        a[0].imshow(best_solution_img)
        a[0].set_title('Best Solution')
        a[1].imshow(face)
        a[1].set_title('Original')
        plt.savefig(f'{save_dir}/{currIteration+1}.png')
        plt.close(fig)

### 10. Regenerating the image

In [15]:
faceVector = img2vector(face)/255
population = initialPopulation((M,N))
flush_dir('image')
for iteration in range(250):
    qualities = calPopFitness(faceVector, population)
    print('Quality: ', qualities)
    print('Iteration : ', iteration)
    parents = getBestParents(population, qualities, num_parents_mating)
    population = crossover(parents, (M,N), nIndividuals = sol_per_pop)
    population = mutation(population= population,numParentsMating=num_parents_mating,mutPercent=mutation_percent)
    saveImages(iteration, qualities,population, (M,N), save_point= 1, save_dir= os.path.join(os.curdir,'image'))

Quality:  [-53. -55. -48. -46. -52. -52. -48. -52.]
Iteration :  0
Index of best -[3 6 2 7 5 4 0 1]
Best qualites -[-46. -48. -48. -52. -52. -52. -53. -55.]
Quality:  [-46. -48. -48. -48. -47. -47. -48. -47.]
Iteration :  1
Index of best -[0 7 5 4 6 3 2 1]
Best qualites -[-46. -47. -47. -47. -48. -48. -48. -48.]


Quality:  [-46. -47. -47. -47. -48. -47. -45. -45.]
Iteration :  2
Index of best -[7 6 0 5 3 2 1 4]
Best qualites -[-45. -45. -46. -47. -47. -47. -47. -48.]
Quality:  [-45. -45. -46. -45. -47. -45. -45. -46.]
Iteration :  3
Index of best -[6 5 3 1 0 7 2 4]
Best qualites -[-45. -45. -45. -45. -45. -46. -46. -47.]
Quality:  [-45. -45. -45. -46. -45. -46. -46. -44.]
Iteration :  4
Index of best -[7 4 2 1 0 6 5 3]
Best qualites -[-44. -45. -45. -45. -45. -46. -46. -46.]
Quality:  [-44. -45. -45. -43. -46. -44. -44. -46.]
Iteration :  5
Index of best -[3 6 5 0 2 1 7 4]
Best qualites -[-43. -44. -44. -44. -45. -45. -46. -46.]
Quality:  [-43. -44. -44. -46. -43. -42. -43. -45.]
Iteration :  6
Index of best -[5 6 4 0 2 1 7 3]
Best qualites -[-42. -43. -43. -43. -44. -44. -45. -46.]
Quality:  [-42. -43. -43. -42. -42. -42. -42. -42.]
Iteration :  7
Index of best -[7 6 5 4 3 0 2 1]
Best qualites -[-42. -42. -42. -42. -42. -42. -43. -43.]
Quality:  [-42. -42. -42. -43. -43. -43. -41. -43.]
Iterat

### 11. Visualization

In [16]:
folder_path = 'image'
file_names = sorted([f for f in os.listdir(folder_path) if f.endswith('.png')], key=natural_sort_key)
mp4_file = 'evolution.mp4'
with imageio.get_writer(mp4_file, fps=5) as writer:
    for filename in file_names:
        image = imageio.imread(os.path.join(folder_path, filename))
        writer.append_data(image)
print(f"MP4 created successfully: {mp4_file}")

C:\Users\sachi\AppData\Local\Temp\ipykernel_18832\1592571157.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(os.path.join(folder_path, filename))


MP4 created successfully: evolution.mp4
